In [1]:
import pandas as pd
import numpy as np
from IPython.display import display
from IPython.display import HTML
#from skspatial.measurement import area_signed
import itertools

# Set display options
pd.options.display.max_columns=None

# Test on 1 game
- The entire map size is 16,000 x 16,000, center at (8000, 8000)

In [2]:
# Read in 1 game
filepath = "./games/lck_spring_2023/"
filename = "ESPORTSTMNT03_3098196.gz"
filegz = filepath + filename
df = pd.read_json(filegz, compression="gzip")

In [3]:
df.head()

,eventTime,eventType,platformGameId,participants,stageID,sequenceIndex,gameName,gameVersion,statsUpdateInterval,playbackID,gameTime,spawnTime,monsterName,nextDragonSpawnTime,nextDragonName,gameOver,teams,itemID,participantID,skillSlot,participant,evolved,wardType,position,placer,killer,bountyGold,killType,assistants,monsterType,inEnemyJungle,killerTeamID,level,maxCooldown,summonerSpellName,summonerSpellSlot,victimTeamID,victim,shutdownBounty,killStreakLength,bounty,dragonType,goldGain,itemBeforeUndo,lastHitter,teamID,lane,buildingType,turretTier,itemsAdded,wallTime,winningTeam
0,2023-02-02T08:06:49.492Z,game_info,ESPORTSTMNT03:3098196,"[{'keystoneID': 8008, 'hashedIP': '3VoTUXLOxMy...",1,0,109625152897595358|game1,13.1.487.8994,1000.0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2023-02-02T08:06:49.492Z,queued_epic_monster_info,ESPORTSTMNT03:3098196,NaN,1,1,109625152897595358|game1,NaN,NaN,1,0.0,1200.0,Baron,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2023-02-02T08:06:49.492Z,queued_epic_monster_info,ESPORTSTMNT03:3098196,NaN,1,2,109625152897595358|game1,NaN,NaN,1,0.0,480.0,RiftHerald,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2023-02-02T08:07:02.337Z,queued_dragon_info,ESPORTSTMNT03:3098196,NaN,1,14,109625152897595358|game1,NaN,NaN,1,164.0,NaN,NaN,300.0,chemtech,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2023-02-02T08:07:02.337Z,queued_epic_monster_info,ESPORTSTMNT03:3098196,NaN,1,15,109625152897595358|game1,NaN,NaN,1,164.0,300.0,DragonChemtech,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
# Split df into 5 min intervals
df["eventTime"] = pd.to_datetime(df["eventTime"])
# df_intervals is array of dataframes in 5min intervals
df_intervals =  [g for n, g in df.groupby(pd.Grouper(key='eventTime',freq='5Min'))]

In [6]:
def get_win_lose_team():
    win_lose_dict = dict()
    # Get the winning team from last row
    win_team = int(df.iloc[[-1]].winningTeam.item())

    if win_team == 200:
        win_lose_dict["lose"] = 100
    else:
        win_lose_dict["lose"] = 200

    win_lose_dict["win"] = win_team
    return win_lose_dict
    
get_win_lose_team()

{'lose': 100, 'win': 200}

## Getting player positions

In [7]:
# Test on 1 file
part_df = df["participants"].dropna().reset_index(drop=True)
# NEED TO ITERATE OVER ALL ROWS
part_df_row = part_df.iloc[1929]

blue_players_filter = filter(lambda x: x["teamID"] == 100, part_df_row)
blue_players = list(blue_players_filter)
blue_positions_map = map(lambda p: (p["position"]["x"], p["position"]["z"]), blue_players)
blue_positions = list(blue_positions_map)
blue_positions

[(4000, 602), (2139, 677), (642, 287), (396, 520), (3050, 656)]

In [47]:
# The entire map size is 16,000 x 16,000, center at (8000, 8000)

In [5]:
# AREA based on radius 1000
PROXIMITY_AREA = 3141592.65359

def calculate_area(point1, point2, point3):
    x1, y1 = point1
    x2, y2 = point2
    x3, y3 = point3
    area = 0.5 * (x1 * (y2 - y3) + x2 * (y3 - y1) + x3 * (y1 - y2))
    return area

# Get # of instances where team members were in close proximity
def calculate_proximity(positions):    
    # Generate all combinations of 3 points
    combinations = itertools.combinations(positions, 3)
    
    triangle_count = 0
    for combo in combinations:
        p1, p2, p3 = combo
        # Check if the points are not collinear (non-zero area)+
        area = calculate_area(p1, p2, p3)
        if area > 0 and area <= PROXIMITY_AREA:
            triangle_count += 1
        # print(f"Points {p1}, {p2}, {p3}, Area = {area}")

    return triangle_count

In [78]:
calculate_proximity(blue_positions)

4

## Get proximity for each interval

Proximity is the % of time the team was in close proximity within the 5min interval. 

(num of times in close prox / data points)

In [12]:
def get_team_proximities(part_df_row, team):
    
    team_players_filter = filter(lambda x: x["teamID"] == team, part_df_row)
    team_players = list(team_players_filter)

    # Check if position data exists
    if "position" not in team_players[0].keys():
        return 0
    
    team_positions_map = map(lambda p: (p["position"]["x"], p["position"]["z"]), team_players)
    team_positions = list(team_positions_map)
    return calculate_proximity(team_positions)

def get_proximities(interval):
    team_prox = dict()
    
    team_dict = get_win_lose_team()
    win_team = team_dict["win"]
    lose_team = team_dict["lose"]

    
    part_df = interval["participants"].dropna().reset_index(drop=True)
    # NEED TO ITERATE OVER ALL ROWS

    win_prox = part_df.apply(get_team_proximities, team=win_team)
    lose_prox = part_df.apply(get_team_proximities, team=lose_team)
    print(win_prox.sum())
    print(lose_prox.sum())

    team_prox["win"] = win_prox.sum() / len(win_prox)
    team_prox["lose"] = lose_prox.sum() / len(lose_prox)

    return team_prox

#get_proximities(df_intervals[0])
#test = df_intervals["participants"].apply(get_proximities)
test = map(get_proximities, df_intervals)
list(test)

576
341
634
576
819
731
730
879
879
859
1033
1195
1039
830


[{'win': 3.6, 'lose': 2.13125},
 {'win': 2.1133333333333333, 'lose': 1.92},
 {'win': 2.73, 'lose': 2.4366666666666665},
 {'win': 2.433333333333333, 'lose': 2.93},
 {'win': 2.93, 'lose': 2.8633333333333333},
 {'win': 3.4433333333333334, 'lose': 3.9833333333333334},
 {'win': 3.791970802919708, 'lose': 3.0291970802919708}]

In [37]:
df_intervals[len(df_intervals) - 1].winningTeam

2832      NaN
2833      NaN
2834      NaN
2835      NaN
2836      NaN
        ...  
3338      NaN
3339      NaN
3340      NaN
3341      NaN
3342    200.0
Name: winningTeam, Length: 511, dtype: float64